In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from mymodule import *

In [2]:
gicode = '000270'
# get_stock_items(gicode)

In [3]:
SRIM(gicode)

{'지배주주지분': 393379,
 '유통주식수': '395,081,692주',
 'ROE': 19.33,
 '기준ROE': '예상 ROE',
 '할인율': 10.85,
 '기업가치': '700831억원',
 '적정주가': '177,389원',
 '이익_지속': 177389,
 '10%_감소': 136016,
 '20%_감소': 121465,
 'flag': True}

In [3]:
MPER(gicode)

{'영업이익추정합': '14,105억원', '적정주가': 936949, 'flag': True}